In [ ]:
import cv2
import os
import glob
import sys
import tensorflow as tf

from scipy import misc
import numpy as np

from tensorflow.contrib.keras.python import keras
from tensorflow.contrib.keras.python.keras import layers, models

from tensorflow import image

from utils import scoring_utils
from utils.separable_conv2d import SeparableConv2DKeras, BilinearUpSampling2D
from utils import data_iterator
from utils import plotting_tools 
from utils import model_tools

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
img_dir = os.path.abspath('./images/')

In [ ]:
data_dir = "../data/train"
mask_dir = "../data/train/masks"
img_dir = "../data/train/images"
mask_path = os.path.join(mask_dir, "*.png")
img_path = os.path.join(img_dir, "*.png")

In [ ]:
# Check the percent of hero pictures 
total_files = 0
total_hero = 0
total_num = 0

for file in glob.glob(mask_path):
    total_files +=1
    img = cv2.imread(file)
    blue = img[:,:,0]
    green = img[:,0,:]

    if np.any(blue == 255):
        total_hero += 1
    if np.any(blue == 255) or np.any(green == 255):
        total_num += 1

percent_hero = 100. * total_hero / total_files
percent_num = 100. * total_num / total_files

print (percent_hero, "percent of files contain the hero")
print(percent_num, "percent of files contain people")

In [ ]:
# Delete images without people


In [ ]:
# data cleaner masks images

# 0_run2_cam1_00000.jpeg
# 0_run2_mask_00000.png
total_num = 0
file_num = 0
for file in glob.glob(mask_path):
    total_num += 1
    img = cv2.imread(file)
    red = img[0,:,:]
    blue = img[:,:,0]
    green = img[:,0,:]
    
    if not (np.any(blue>50) or np.any(green>50)):
        file_num += 1
        
print(file_num, total_num)


In [1]:
# Load resnet152 model 
from keras.models import Model
from keras import layers
from keras.layers import Input, Dense, Conv2D, Dropout, Activation
from keras.layers import MaxPooling2D, AveragePooling2D, Flatten, merge, Reshape #,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from utils.scale_layer import Scale
from keras.optimizers import SGD

Using TensorFlow backend.


In [ ]:
resnet152_dir = "../data/weights/resnet152_weights_tf.h5"
resnet152 = Model.load_weights(filepath=resnet152_dir, by_name=True)

In [4]:
img_rows = 256
img_cols = 256
num_channel = 3
num_classes = 3
image_shape = (img_rows, img_cols, num_channel)


model = resnet152_model(img_rows, img_cols, num_channel, num_classes)

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, name="conv1", strides=2, padding="same", use_bias=False, kernel_size=7)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, strides=(1, 1), name="res2a_branch2a", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:61: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, name="res2a_branch2b", strides=1, padding="same", use_bias=False, kernel_size=3)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:66: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, strides=1, name="res2a_branch2c", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:71: UserWarn

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), name="res3b7_branch2a", use_bias=False)`
  app.launch_new_instance()
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, name="res3b7_branch2b", strides=1, padding="same", use_bias=False, kernel_size=3)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, strides=1, name="res3b7_branch2c", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, strides=(2, 2), name="res4a_branch2a", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:61:

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), name="res4b11_branch2a", use_bias=False)`
  app.launch_new_instance()
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, name="res4b11_branch2b", strides=1, padding="same", use_bias=False, kernel_size=3)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, strides=1, name="res4b11_branch2c", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), name="res4b12_branch2a", use_bias=False)`
  app.launch_new_instance()
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_laun

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), name="res4b23_branch2a", use_bias=False)`
  app.launch_new_instance()
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, name="res4b23_branch2b", strides=1, padding="same", use_bias=False, kernel_size=3)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, strides=1, name="res4b23_branch2c", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), name="res4b24_branch2a", use_bias=False)`
  app.launch_new_instance()
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_laun

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (1, 1), name="res4b35_branch2a", use_bias=False)`
  app.launch_new_instance()
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, name="res4b35_branch2b", strides=1, padding="same", use_bias=False, kernel_size=3)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1024, strides=1, name="res4b35_branch2c", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, strides=(2, 2), name="res5a_branch2a", use_bias=False, kernel_size=1)`
D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel_launcher.py

In [2]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    '''The identity_block is the block that has no conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, 1, 1, name=conv_name_base + '2a', bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

#     x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, kernel_size=kernel_size, strides=1, padding='same',
                      name=conv_name_base + '2b', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, kernel_size=1, strides=1, name=conv_name_base + '2c', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = merge([x, input_tensor], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2,2)):
    '''conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should have subsample=(2,2) as well
    '''
    eps = 1.1e-5
    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, kernel_size=1, strides=strides,
                      name=conv_name_base + '2a', bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

#     x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, kernel_size=kernel_size, strides=1, padding='same',
                      name=conv_name_base + '2b', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, kernel_size=1, strides=1, name=conv_name_base + '2c', bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, kernel_size=1, strides=strides,
                             name=conv_name_base + '1', bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = merge([x, shortcut], mode='sum', name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x

In [3]:
def resnet152_model(img_rows, img_cols, color_type=1, num_classes=None):
    """
    Resnet 152 Model for Keras
    Model Schema and layer naming follow that of the original Caffe implementation
    https://github.com/KaimingHe/deep-residual-networks
    ImageNet Pretrained Weights 
    Theano: https://drive.google.com/file/d/0Byy2AcGyEVxfZHhUT3lWVWxRN28/view?usp=sharing
    TensorFlow: https://drive.google.com/file/d/0Byy2AcGyEVxfeXExMzNNOHpEODg/view?usp=sharing
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
    """
    eps = 1.1e-5

    # Handle Dimension Ordering for different backends
    global bn_axis
    
    bn_axis = 3
    img_input = Input(shape=(img_rows, img_cols, color_type), name='data')
#     x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', name='conv1', bias=False)(img_input)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1,8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b'+str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1,36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b'+str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x_fc = AveragePooling2D((7, 7), name='avg_pool')(x)
    x_fc = Flatten()(x_fc)
    x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)

    model = Model(img_input, x_fc)

    # Use pre-trained weights for Tensorflow backend
    weights_path = "../data/weights/resnet152_weights_tf.h5"

    model.load_weights(weights_path, by_name=True)

    # Truncate and replace softmax layer for transfer learning
    # Cannot use model.layers.pop() since model is not of Sequential() type
    # The method below works since pre-trained weights are stored in layers but not in the model
    x_newfc = AveragePooling2D((7, 7), name='avg_pool')(x)
    x_newfc = Flatten()(x_newfc)
    x_newfc = Dense(num_classes, activation='softmax', name='fc8')(x_newfc)

    model = Model(img_input, x_newfc)

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model